# ***Grammar Error Correction***

# *Downloading required packages*

In [22]:
!pip install datasets
!pip install autocorrect
!pip install transformers
!pip install tensorflow
!pip install numpy
!pip install pandas 
!pip install sklearn

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
       

# *Importing packages*

In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AdamW,T5ForConditionalGeneration,T5Tokenizer
from sklearn.model_selection import train_test_split
from autocorrect import Speller

# *Setting seeding*

In [7]:
def set_seed(seed):
  tf.random.set_seed(seed)
  np.random.seed(seed)

set_seed(42)

# *Loading C4 200M dataset*

In [8]:
pd.set_option('display.max_colwidth', None)

column_names = ['input','output']
df = pd.read_csv('/kaggle/input/c4200m/C4_200M.tsv-00000-of-00010',sep = '\t',names=column_names, header=None)
df = df.drop(df.index[0])

In [9]:
df = df[:10000]
df.head()

,input,output
1,The effect of widespread dud targets two face up attack position monsters on the field.,"1. The effect of ""widespread dud"" targets two face up attack position monsters on the field."
2,"tax on sales of stores for non residents are set at 21% for 2014 and 20% in 2015 payable on sales tentatively earned from the difference of the property value some time of purchase (price differences according to working time) and theyear to which sale couples (sales costs), based on the approved annual on the base approved by law).","Capital Gains tax on the sale of properties for non-residents is set at 21% for 2014 and 20% in 2015 payable on profits earned on the difference of the property value between the year of purchase (purchase price plus costs) and the year of sale (sales price minus costs), based on the approved annual percentage increase on the base value approved by law."
3,Much many brands and sellers still in the market.,Many brands and sellers still in the market.
4,this is is the latest Maintenance release of Samba 3.6,This is is the latest maintenance release of Samba 3.6.
5,"Fairy Or Not, I'm the Godmother: no just look, but my outfit for taking the part as godmother.","Fairy Or Not, I'm the Godmother: Not just a look, but my outfit for taking on the role as godmother."


# *Initializing the model*

In [10]:
from transformers import TFT5ForConditionalGeneration,T5Tokenizer

model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = TFT5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


# *Preprocessing the dataframe*

In [11]:
df.isnull().sum()

input     0
output    0
dtype: int64

In [12]:
df.fillna('', inplace=True)

In [13]:
# Making a list of strings
uncorrected = df['input'].astype(str) 
corrected = df['output'].astype(str)

# Tokenizing the text of every 
tokenized_input = tokenizer(uncorrected.tolist(), padding=True, truncation=True, return_tensors='tf')
tokenized_output = tokenizer(corrected.tolist(), padding=True, truncation=True, return_tensors='tf')

# making list of input_ids and attention masks 
X_input_ids = tokenized_input['input_ids'].numpy()
Y_input_ids = tokenized_output['input_ids'].numpy()
X_attention_mask = tokenized_input['attention_mask'].numpy()
Y_attention_mask = tokenized_output['attention_mask'].numpy()

In [14]:
from sklearn.model_selection import train_test_split

# Splitting the numpy array into train, validation and test of inputs and outputs 
X_train, X_test, Y_train, Y_test = train_test_split(X_input_ids, Y_input_ids, test_size=0.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(X_train,Y_train,test_size = 0.1, random_state = 42)
X_train_attention_mask, X_test_attention_mask, Y_train_attention_mask, Y_test_attention_mask = train_test_split(X_attention_mask, Y_attention_mask, test_size=0.1, random_state=42)
x_train_attention_mask, x_val_attention_mask, y_train_attention_mask, y_val_attention_mask = train_test_split(X_train_attention_mask, Y_train_attention_mask, test_size=0.1, random_state=42)

# Making a input model directory 
model_input = {
    'input_ids': x_train,
    'attention_mask': x_train_attention_mask,
    'labels': y_train
}


# *Compiling and Training the model*

In [15]:
import tensorflow as tf


strategy = tf.distribute.MirroredStrategy()   # used for running the processes simulteneously on multiple GPUs
with strategy.scope():
    model_name = 't5-small' 
    model = TFT5ForConditionalGeneration.from_pretrained(model_name)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(          # Defining a loss function
        from_logits=True, reduction=tf.keras.losses.Reduction.SUM     # sums up the losses      
    )
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy']) # compling 

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


# *Making 3 datasets : Train, test and validation*

In [16]:
batch_size = 16 # defining the size of one batch

train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': model_input['input_ids'],
        'attention_mask': model_input['attention_mask'],
        'labels': model_input['labels']
    }
)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': X_test,
        'attention_mask': X_test_attention_mask,
        'labels': Y_test
    }
)).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': X_test,
        'attention_mask': X_test_attention_mask,
        'labels': Y_test
    }
)).batch(batch_size)

# *Training the model*

In [17]:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


I0000 00:00:1722074609.398904     136 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


507/507 [==============================] - 467s 741ms/step - loss: 886.8325 - accuracy: 0.9760 - val_loss: 410.2971 - val_accuracy: 0.9853
Epoch 2/3
507/507 [==============================] - 357s 705ms/step - loss: 417.7831 - accuracy: 0.9848 - val_loss: 401.5069 - val_accuracy: 0.9857
Epoch 3/3
507/507 [==============================] - 358s 706ms/step - loss: 365.7963 - accuracy: 0.9861 - val_loss: 404.8884 - val_accuracy: 0.9858


# *Testing*

In [18]:
test_loss, test_accuracy = model.evaluate(test_dataset) 
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

63/63 [==============================] - 17s 262ms/step - loss: 404.8884 - accuracy: 0.9858
Test loss: 404.888427734375
Test accuracy: 0.9858379364013672


# *Spelling correction*

In [19]:
from autocorrect import Speller

spell = Speller(lang='en')

def correct_spelling(text):
    return spell(text)

# *Generating output function*

In [20]:
def generate_output(model, tokenizer, input_text):
    input_tokens = tokenizer.encode(input_text, return_tensors='tf')

    output_tokens = model.generate(input_ids=input_tokens, max_length=50)

    output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    return correct_spelling(output_text)

input_text = input()
output_text = generate_output(model, tokenizer, input_text)
print("Input Text: ", input_text)
print("Output Text: ", output_text)

 I is playing football and I believe that I would be the better at it.


Input Text:  I is playing football and I believe that I would be the better at it.
Output Text:  I am playing football and I believe that I would be the better at it.


# Saving the model for future use

In [23]:
model.save_weights('GEC.h5')